In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from math import sqrt

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import feature_selection


#import lightgbm as lgb

#from sklearn.linear_model import LinearRegression

#import warnings
#warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
data = pd.read_csv('Data/tcd-ml-1920-group-income-train.csv', low_memory = False)
dataTest  = pd.read_csv('Data/tcd-ml-1920-group-income-test.csv', low_memory = False)

In [3]:
del_col_list = ['Instance','Wears Glasses']
data = data.drop(del_col_list, axis=1)
#data= data[data["Income"]>0]
#data.info()
#data["Income"] = np.log(data["Income"])

data_test_original = dataTest
del_col_list = ['Instance', 'Wears Glasses']
dataTest = dataTest.drop(del_col_list, axis=1)



In [4]:
print("Old Size : " , data.shape[0])
print("Duplicates: " , data.duplicated().sum())
data.drop_duplicates(inplace=True) 
print("New Size : " , data.shape[0])

Old Size :  1048574
Duplicates:  144555
New Size :  904019


In [5]:
'''year_max_value = data['Size of City'].max()
year_min_value = data['Size of City'].min()
data['Size of City'] = (data['Size of City'] - year_min_value) / (year_max_value - year_min_value)

year_max_value = dataTest['Size of City'].max()
year_min_value = dataTest['Size of City'].min()
dataTest['Size of City'] = (dataTest['Size of City'] - year_min_value) / (year_max_value - year_min_value)'''

"year_max_value = data['Size of City'].max()\nyear_min_value = data['Size of City'].min()\ndata['Size of City'] = (data['Size of City'] - year_min_value) / (year_max_value - year_min_value)\n\nyear_max_value = dataTest['Size of City'].max()\nyear_min_value = dataTest['Size of City'].min()\ndataTest['Size of City'] = (dataTest['Size of City'] - year_min_value) / (year_max_value - year_min_value)"

In [6]:
data[["Year of Record"]] = data[["Year of Record"]].fillna(value=data["Year of Record"].mode()[0])

dataTest[["Year of Record"]] = dataTest[["Year of Record"]].fillna(value=dataTest["Year of Record"].mode()[0])

data[["Hair Color"]].fillna(value="unknownHC")
data[['Hair Color']] = data[['Hair Color']].replace('0', 'unknownHC') 
data[['Hair Color']] = data[['Hair Color']].replace('Unknown', 'unknownHC')
data[['Hair Color']] = data[['Hair Color']].replace('nan', 'unknownHC')


data[["Gender"]].fillna(value="unknownG")
data[['Gender']] = data[['Gender']].replace('0', 'unknownG') 
data[['Gender']] = data[['Gender']].replace('unknown', 'unknownG')

data[["University Degree"]] = data[["University Degree"]].fillna(data["University Degree"].mode()[0])

data[["Profession"]] = data[["Profession"]].fillna(value="unknownP")

data[['Satisfation with employer']] = data[['Satisfation with employer']].fillna(data["Satisfation with employer"].mode()[0])




dataTest[["Hair Color"]].fillna(value="unknownHC")
dataTest[['Hair Color']] = dataTest[['Hair Color']].replace('0', 'unknownHC') 
dataTest[['Hair Color']] = dataTest[['Hair Color']].replace('Unknown', 'unknownHC')
dataTest[['Hair Color']] = dataTest[['Hair Color']].replace('nan', 'unknownHC')

dataTest[["University Degree"]] = dataTest[["University Degree"]].fillna(dataTest["University Degree"].mode()[0])

dataTest[["Profession"]] = dataTest[["Profession"]].fillna(value="unknownP")

dataTest[["Gender"]] = dataTest[["Gender"]].fillna(value="unknownG")
dataTest[['Gender']] = dataTest[['Gender']].replace('0', 'unknownG') 
dataTest[['Gender']] = dataTest[['Gender']].replace('unknown', 'unknownG')

dataTest[['Satisfation with employer']] = dataTest[['Satisfation with employer']].fillna(data["Satisfation with employer"].mode()[0])
dataTest[["Country"]] = dataTest[["Country"]].fillna(dataTest["Country"].mode()[0])

In [7]:
'''def create_cat_con(df,cats,cons,normalize=True):
    for i,cat in enumerate(cats):
        vc = df[cat].value_counts(dropna=False, normalize=normalize).to_dict()
        nm = cat + '_FE_FULL'
        df[nm] = df[cat].map(vc)
        df[nm] = df[nm].astype('float32')
        for j,con in enumerate(cons):
#             print("cat %s con %s"%(cat,con))
            new_col = cat +'_'+ con
            #print('timeblock frequency encoding:', new_col)
            df[new_col] = df[cat].astype(str)+'_'+df[con].astype(str)
            temp_df = df[new_col]
            fq_encode = temp_df.value_counts(normalize=True).to_dict()
            df[new_col] = df[new_col].map(fq_encode)
            df[new_col] = df[new_col]/df[cat+'_FE_FULL']
    return df'''

'def create_cat_con(df,cats,cons,normalize=True):\n    for i,cat in enumerate(cats):\n        vc = df[cat].value_counts(dropna=False, normalize=normalize).to_dict()\n        nm = cat + \'_FE_FULL\'\n        df[nm] = df[cat].map(vc)\n        df[nm] = df[nm].astype(\'float32\')\n        for j,con in enumerate(cons):\n#             print("cat %s con %s"%(cat,con))\n            new_col = cat +\'_\'+ con\n            #print(\'timeblock frequency encoding:\', new_col)\n            df[new_col] = df[cat].astype(str)+\'_\'+df[con].astype(str)\n            temp_df = df[new_col]\n            fq_encode = temp_df.value_counts(normalize=True).to_dict()\n            df[new_col] = df[new_col].map(fq_encode)\n            df[new_col] = df[new_col]/df[cat+\'_FE_FULL\']\n    return df'

In [8]:
'''data.info()'''

'data.info()'

In [9]:
data['Yearly Income in addition to Salary (e.g. Rental Income)'] = data['Yearly Income in addition to Salary (e.g. Rental Income)'].str[:-3].astype(np.float64)
dataTest['Yearly Income in addition to Salary (e.g. Rental Income)'] = dataTest['Yearly Income in addition to Salary (e.g. Rental Income)'].str[:-3].astype(np.float64)


In [10]:
groupedProf = data.groupby('Profession', as_index=False)['Income'].mean()
data['Profession'] = data['Profession'].map(groupedProf.set_index('Profession')['Income'])

groupedCountry = data.groupby('Country', as_index=False)['Income'].mean()
data['Country'] = data['Country'].map(groupedCountry.set_index('Country')['Income'])

groupedUD = data.groupby('University Degree', as_index=False)['Income'].mean()
data['University Degree'] = data['University Degree'].map(groupedUD.set_index('University Degree')['Income'])

groupedG = data.groupby('Gender', as_index=False)['Income'].mean()
data['Gender'] = data['Gender'].map(groupedG.set_index('Gender')['Income'])

groupedHS = data.groupby('Housing Situation', as_index=False)['Income'].mean()
data['Housing Situation'] = data['Housing Situation'].map(groupedHS.set_index('Housing Situation')['Income'])

groupedWE = data.groupby('Work Experience in Current Job [years]', as_index=False)['Income'].mean()
data['Work Experience in Current Job [years]'] = data['Work Experience in Current Job [years]'].map(groupedWE.set_index('Work Experience in Current Job [years]')['Income'])

groupedSWE = data.groupby('Satisfation with employer', as_index=False)['Income'].mean()
data['Satisfation with employer'] = data['Satisfation with employer'].map(groupedSWE.set_index('Satisfation with employer')['Income'])

#groupedYIA = data.groupby('Yearly Income in addition to Salary (e.g. Rental Income)', as_index=False)['Income'].mean()
#data['Yearly Income in addition to Salary (e.g. Rental Income)'] = data['Yearly Income in addition to Salary (e.g. Rental Income)'].map(groupedYIA.set_index('Yearly Income in addition to Salary (e.g. Rental Income)')['Income'])

groupedHC = data.groupby('Hair Color', as_index=False)['Income'].mean()
data['Hair Color'] = data['Hair Color'].map(groupedHC.set_index('Hair Color')['Income'])






dataTest['Profession'] = dataTest['Profession'].map(groupedProf.set_index('Profession')['Income'])
dataTest['Country'] = dataTest['Country'].map(groupedCountry.set_index('Country')['Income'])
dataTest['University Degree'] = dataTest['University Degree'].map(groupedUD.set_index('University Degree')['Income'])
dataTest['Gender'] = dataTest['Gender'].map(groupedG.set_index('Gender')['Income'])
dataTest['Housing Situation'] = dataTest['Housing Situation'].map(groupedHS.set_index('Housing Situation')['Income'])
dataTest['Work Experience in Current Job [years]'] = dataTest['Work Experience in Current Job [years]'].map(groupedWE.set_index('Work Experience in Current Job [years]')['Income'])
dataTest['Satisfation with employer'] = dataTest['Satisfation with employer'].map(groupedSWE.set_index('Satisfation with employer')['Income'])
dataTest['Hair Color'] = dataTest['Hair Color'].map(groupedHC.set_index('Hair Color')['Income'])


In [11]:
data.corr()

,Year of Record,Housing Situation,Crime Level in the City of Employement,Work Experience in Current Job [years],Satisfation with employer,Gender,Age,Country,Size of City,Profession,University Degree,Hair Color,Body Height [cm],Yearly Income in addition to Salary (e.g. Rental Income),Income
Year of Record,1.000000,0.829247,-0.000782,0.000645,0.000207,0.212794,-0.000180,0.001261,0.000054,0.011949,0.001972,0.002014,0.000222,0.001077,0.641993
Housing Situation,0.829247,1.000000,0.000502,0.000761,-0.000102,0.321798,-0.000051,0.000541,0.000642,0.014237,0.001338,0.001888,0.000745,0.000949,0.746288
Crime Level in the City of Employement,-0.000782,0.000502,1.000000,0.189110,0.092231,0.001130,0.197552,-0.169846,0.000149,0.000049,0.005012,-0.003031,-0.000944,-0.000828,-0.026945
Work Experience in Current Job [years],0.000645,0.000761,0.189110,1.000000,-0.021524,0.004000,0.963008,0.000869,-0.000777,0.090184,0.000243,-0.001170,0.052085,0.000578,0.109329
Satisfation with employer,0.000207,-0.000102,0.092231,-0.021524,1.000000,0.001131,0.001022,0.353173,0.002780,0.022816,0.001980,-0.000774,-0.001446,-0.000548,0.097963
Gender,0.212794,0.321798,0.001130,0.004000,0.001131,1.000000,0.000963,-0.000215,0.000504,0.005212,-0.001137,0.000939,0.032925,0.000227,0.270775
Age,-0.000180,-0.000051,0.197552,0.963008,0.001022,0.000963,1.000000,0.000594,-0.000469,0.003320,-0.000102,-0.001338,0.001038,0.000201,0.103523
Country,0.001261,0.000541,-0.169846,0.000869,0.353173,-0.000215,0.000594,1.000000,-0.000060,0.005349,0.000275,-0.000378,0.000016,0.000388,0.268653
Size of City,0.000054,0.000642,0.000149,-0.000777,0.002780,0.000504,-0.000469,-0.000060,1.000000,-0.000609,-0.000686,0.002653,-0.001116,-0.001410,0.024590
Profession,0.011949,0.014237,0.000049,0.090184,0.022816,0.005212,0.003320,0.005349,-0.000609,1.000000,0.001712,0.001314,0.001748,0.005635,0.075848


In [12]:
'''use_train_size = data.shape[0]
print("Train size: " , use_train_size  )
dataFull = pd.concat([data,dataTest],ignore_index=True)'''

'use_train_size = data.shape[0]\nprint("Train size: " , use_train_size  )\ndataFull = pd.concat([data,dataTest],ignore_index=True)'

In [13]:

X = data.drop('Income', axis = 1)
y = data['Income']

X_train, X_validation, y_train, y_validation = train_test_split(X,y,test_size=0.25, random_state=42)




In [ ]:
model = CatBoostRegressor(eval_metric='MAE')
model.fit(X_train, y_train)
grid = {'learning_rate': [0.1],
        'depth': [8, 10, 12, 14, 16]}

randomized_search_result = model.grid_search(grid,
                                                   X=X_train,
                                                   y=y_train,
                                                   plot=True,
                                                   cv=3,
                                                  partition_random_seed=0,
                                                  calc_cv_statistics=True, 
                                                  search_by_train_test_split=True,
                                                  refit=True, 
                                                  shuffle=True, 
                                                  stratified=None, 
                                                  train_size=0.8, 
                                                  verbose=True)

In [14]:
cb_model = CatBoostRegressor(iterations=3000,
                             learning_rate=0.1,
                             depth=10,
                             eval_metric='MAE',
                             random_seed = 23,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
cb_model.fit(X_train, y_train,
             eval_set=(X_validation,y_validation),
             use_best_model=True,
             verbose=True)

0:	learn: 70175.5167628	test: 70242.2437223	best: 70242.2437223 (0)	total: 196ms	remaining: 9m 49s
75:	learn: 11345.6875668	test: 11435.3462575	best: 11435.3462575 (75)	total: 10.7s	remaining: 6m 52s
150:	learn: 10384.4633998	test: 10556.7049121	best: 10556.7049121 (150)	total: 21.3s	remaining: 6m 41s
225:	learn: 10028.3829853	test: 10286.3504639	best: 10286.3504639 (225)	total: 31.7s	remaining: 6m 29s
300:	learn: 9798.4764781	test: 10149.2515649	best: 10149.2515649 (300)	total: 42.2s	remaining: 6m 18s
375:	learn: 9630.5511678	test: 10074.5980847	best: 10074.5980847 (375)	total: 53.2s	remaining: 6m 11s
450:	learn: 9481.0082915	test: 10015.9829158	best: 10015.9829158 (450)	total: 1m 4s	remaining: 6m 2s
525:	learn: 9344.0908846	test: 9976.3421236	best: 9976.3421236 (525)	total: 1m 15s	remaining: 5m 53s
600:	learn: 9218.4922597	test: 9943.8299927	best: 9943.8299927 (600)	total: 1m 26s	remaining: 5m 43s
675:	learn: 9106.9738381	test: 9915.3926889	best: 9914.3002106 (664)	total: 1m 37s	rema

In [15]:
print(cb_model.get_params())

{'iterations': 3000, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'RMSE', 'od_wait': 100, 'od_type': 'Iter', 'random_seed': 23, 'metric_period': 75, 'eval_metric': 'MAE', 'bagging_temperature': 0.2}


In [16]:

dataTest = dataTest.drop('Income', axis = 1)

In [17]:
y_pred = cb_model.predict(dataTest)
y_pred = y_pred.reshape(-1)

In [18]:
instances = data_test_original['Instance'].to_numpy()
to_print = pd.DataFrame({'Instance': instances, 'Total Yearly Income [EUR]': y_pred})
to_print.to_csv('result.csv', index=False)

In [ ]:
'''
num_l = (2**10 - 5)
params = {
          'max_depth': 20,
          'learning_rate': 0.001,
          "boosting": "dart",
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
            "bagging_fraction" : 0.8,
    "bagging_freq" : 100, 
    "num_leaves" : num_l
         }
trn_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_validation, label=y_validation)
model = lgb.train(params, trn_data, 150000, valid_sets = [trn_data, val_data], verbose_eval=1000)
'''

In [ ]:
'''

#dataTest[["Country"]] = dataTest[["Country"]].fillna(value=dataTest["Country"].mean())
#dataTest[["Profession"]] = dataTest[["Profession"]].fillna(value=dataTest["Profession"].mean())'''